In [1]:
#Import Dependencies
import pandas as pd
from pandas.io.json import json_normalize
import tweepy
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
import csv
import multiprocessing
import time
from datetime import datetime

# Import the necessary package to process data in JSON format
try:
    import json
except ImportError:
    import simplejson as json

#Get system time for the current run
now = datetime.now()
current_time = now.strftime("%m_%d_%y__%H_%M")
print("Current Time = ", current_time)

#Function to flatten Json
def flatten_json(nested_json):
    #print("Flattening the Json data for processing convenience...")
    out = {}
    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x
    flatten(nested_json)
    #print("Flattening completed...")
    return out

#Twitter Developer Access Tokens
access_token = "846302774-pPB0TfVkUWOO4aEr8hKNlKupkog1g1VFPnho1Td3"
access_token_secret = "IUafuVm1YOk3S3hIflkUPxbI3Zb6pQubm2vseeBIRAx79"
consumer_key = "uAMKZaMyqwAan03Cd5ZJpL56A"
consumer_secret = "o9Q5OuYx6imBUcYvEIi8MVQ9WdSuCkCGIT9ieA0YNTJJZMBleL"

#Output Listener
class StdOutListener(StreamListener):
    def on_data(self, data):
        #print(data)
        dictList.append(flatten_json(json.loads(data)))
        return True
    def on_error(self, status):
        print(status)

Current Time =  03_25_20__11_57


In [2]:
#Global Variables
dictList = []

#Main Function
if __name__ == '__main__':
    
    #Streaming API
    l = StdOutListener()
    
    #This handles Twitter authetification and the connection to Twitter
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    topics = ['Politics', 'Corona', 'Sports']
    #topics = ['Politics', 'Corona', 'Sports', 'Nutrition', 'Technology']
    for topic in topics:
        if len(dictList) > 0:
            print("Clearing previously streamed data...")
            dictList = []

        try:
            print("Starting to stream tweets for "+topic+" from twitter... Task Starting Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))
            print(("Current size of the data collected: %d")%(len(dictList)))
            stream = Stream(auth, l)

            #Filter Tweets to specified words
            iterator = stream.filter(track=[topic])
        except:
            print(("Twitter Data Collection Inturrupted! Total Tweets Collected: %d")%(len(dictList)))
            print("Task Ending Time: " + datetime.now().strftime("%m/%d/%y %H:%M") + "\n")
            
        #Convert JSON list to dataframe
        print("Converting JSON list to dataframe..." + datetime.now().strftime("%m/%d/%y %H:%M"))
        dfItem = pd.DataFrame.from_records(dictList)
        print("Task Ending Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))
        
        #Get all column names
        allCols = list(dfItem.columns.values)

        #Required Columns from the stream
        cols = ['created_at', 'id_str', 'text', 'user_name', 'user_screen_name',  'user_location', 'retweeted_status_text', 
                'extended_tweet_full_text', 'retweeted_status_extended_tweet_full_text', 'lang']

        #Drop unwanted columns from the dataframe
        print("Creating a dataframe with only required columns from the streamed data... Task Starting Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))
        dropCols = []
        for colName in allCols:
            if colName not in cols:
                dropCols.append(colName)

        dfItem.drop(dropCols, axis=1, inplace=True)
        print("Task Ending Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))

        #Add a dummy string prefix to all the values of the column 'id_str' for computational convenience
        '''print("Adding a dummy string prefix to all the values of the column id_str for computational convenience... Task Starting Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))
        for i in range(len(dfItem)):
            dfItem['id_str'][i] = str(dfItem['id_str'][i]) + 'ABC'
            dfItem['user_id_str'][i] = str(dfItem['user_id_str'][i]) + 'ABC'
            dfItem['in_reply_to_status_id_str'][i] = str(dfItem['in_reply_to_status_id_str'][i]) + 'ABC'
            dfItem['in_reply_to_user_id_str'][i] = str(dfItem['in_reply_to_user_id_str'][i]) + 'ABC'
        print("Task Ending Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))'''

        #Write dataframe to CSV
        print("Creating CSV with the data...")
        dfItem.to_csv("D:/MS COMPUTER SCIENCE/MS PROJECTS/MAIN PROJECT/DataCollection/Twitter/"+topic+"/TwitterData_"+topic+"_"+current_time+".csv", index=False)
        print("Created CSV with the data!")
        print("-------------------------------------------------------\n")

Starting to stream tweets for Politics from twitter... Task Starting Time: 03/25/20 11:57
Current size of the data collected: 0
Twitter Data Collection Inturrupted! Total Tweets Collected: 10754
Task Ending Time: 03/25/20 12:23

Converting JSON list to dataframe...03/25/20 12:23
Task Ending Time: 03/25/20 12:23
Creating a dataframe with only required columns from the streamed data... Task Starting Time: 03/25/20 12:23
Task Ending Time: 03/25/20 12:23
Creating CSV with the data...
Created CSV with the data!
-------------------------------------------------------

Clearing previously streamed data...
Starting to stream tweets for Corona from twitter... Task Starting Time: 03/25/20 12:23
Current size of the data collected: 0
Twitter Data Collection Inturrupted! Total Tweets Collected: 393
Task Ending Time: 03/25/20 12:24

Converting JSON list to dataframe...03/25/20 12:24
Task Ending Time: 03/25/20 12:24
Creating a dataframe with only required columns from the streamed data... Task Starti

In [ ]:
#Drop unwanted columns from the dataframe
print("Creating a dataframe with only required columns from the streamed data... Task Starting Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))
dropCols = []
for colName in allCols:
    if colName not in cols:
        dropCols.append(colName)

dfItem.drop(dropCols, axis=1, inplace=True)
print("Task Ending Time: " + datetime.now().strftime("%m/%d/%y %H:%M"))


In [5]:
print(len(dfItem))

41797


In [ ]:
#Convert JSON list to dataframe
print("Converting JSON list to dataframe..." + datetime.now().strftime("%m/%j/%y %H:%M"))
dfItem = pd.DataFrame.from_records(dictList)
print("Task Ending Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))

In [ ]:
print(max(dfItem['reply_count']))

In [ ]:
    #Convert JSON list to dataframe
    print("Converting JSON list to dataframe..." + datetime.now().strftime("%m/%j/%y %H:%M"))
    dfItem = pd.DataFrame.from_records(dictList)
    print("Task Ending Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))

    #Get all column names
    allCols = list(dfItem.columns.values)

    #Required Columns from the stream
    cols = ['created_at', 'id_str', 'text', 'user_id_str', 'user_name', 'user_screen_name',  'user_verified', 'user_followers_count', 'user_friends_count', 
            'user_favourites_count', 'user_statuses_count', 'in_reply_to_status_id_str', 'in_reply_to_user_id_str',  'in_reply_to_screen_name', 
            'extended_tweet_full_text', 'retweeted_status_extended_tweet_full_text', 'quote_count', 'reply_count', 'retweet_count', 'favorite_count', 
            'favorited', 'retweeted', 'filter_level', 'lang']

    #Drop unwanted columns from the dataframe
    print("Creating a dataframe with only required columns from the streamed data... Task Starting Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))
    dropCols = []
    for colName in allCols:
        if colName not in cols:
            dropCols.append(colName)

    dfItem.drop(dropCols, axis=1, inplace=True)
    print("Task Ending Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))

    #Add a dummy string prefix to all the values of the column 'id_str' for computational convenience
    print("Adding a dummy string prefix to all the values of the column id_str for computational convenience... Task Starting Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))
    for i in range(len(dfItem)):
        dfItem['id_str'][i] = str(dfItem['id_str'][i]) + 'ABC'
        dfItem['user_id_str'][i] = str(dfItem['user_id_str'][i]) + 'ABC'
        dfItem['in_reply_to_status_id_str'][i] = str(dfItem['in_reply_to_status_id_str'][i]) + 'ABC'
        dfItem['in_reply_to_user_id_str'][i] = str(dfItem['in_reply_to_user_id_str'][i]) + 'ABC'
    print("Task Ending Time: " + datetime.now().strftime("%m/%j/%y %H:%M"))

    #Write dataframe to CSV
    print("Creating CSV with the data...")
    dfItem.to_csv("D:/MS COMPUTER SCIENCE/MS PROJECTS/MAIN PROJECT/DataCollection/Twitter/Test/TwitterData_"+current_time+".csv", index=False)
    print("Created CSV with the data...")